In [ ]:
# Setting env
%env TMP_DIR=/home/ubuntu/tmp
%env AWS_REGION=us-west-2
%env AWS_NO_SIGN_REQUEST=YES
%env LOG_LEVEL=DEBUG

In [2]:
import earth_data_kit as edk
import datetime

In [ ]:
edk.__version__

In [4]:
# Initialize the modis dataset using the S3 engine.
source = "s3://modis-pds/MCD43A4.006/{h}/{v}/%Y%j/*_B0?.TIF"
dataset_id = "modis-pds"
engine = "s3"

ds = edk.stitching.Dataset(dataset_id, source, engine, clean=True)

In [ ]:
print (ds)

In [6]:
# Set the temporal bounds for the dataset (e.g., using January 2017 as an example)
start_date = datetime.datetime(2017, 1, 1)
end_date = datetime.datetime(2017, 1, 7)
ds.set_timebounds(start_date, end_date)

In [7]:
# Setting spatial bounds for the dataset

# Specify the grid file that maps the dataset's grid system to world coordinates (e.g., a KML file)
grid_fp = "/home/ubuntu/notebooks/data/modis.kml"
bbox = (19.3044861183, 39.624997667, 21.0200403175, 42.6882473822)
# bbox_india = (68.1766451354, 7.96553477623, 97.4025614766, 35.4940095078)

# Define a function to extract grid components (horizontal 'h' and vertical 'v') from a grid file row.
def extract_grid_components(row):
    import re
    match = re.search(r"h:(\d+)\s+v:(\d+)", row.Name)
    if match:
        return {"h": f"{int(match.group(1)):02d}", "v": f"{int(match.group(2)):02d}"}
    return {}

ds.set_spacebounds(bbox, grid_fp, extract_grid_components)

In [ ]:
print (ds)

In [ ]:
# Running discover to get the bands available in the dataset, you can also set gdal options if needed
# ds.set_gdal_options(["-srcnodata 32767"])
ds.discover()

In [ ]:
# Get the bands discovered in the dataset
ds.get_bands()

In [ ]:
# Stitches the scene files into VRTs using the defined band arrangement.
ds.to_vrts(bands=["Nadir_Reflectance_Band3", "Nadir_Reflectance_Band4"])

In [31]:
# This returns a dataarray with the stitched bands
da = ds.to_dataarray()

In [ ]:
da

In [ ]:
da.edk.plot(time='2017-01-01', band=1)